# Dataset Loading and Preparation


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('/content/selected_features.csv')

# Define features (X) and target variable (y)
X = df.drop(columns=['Close'])  # Drop the target column
y = df['Close']  # Target variable

# Split data into training and testing sets (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features for LSTM and ANN
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))


# 1. Linear Regression


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold

# Initialize Linear Regression
lr_model = LinearRegression()

# Perform 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
lr_scores_mse = -cross_val_score(lr_model, X, y, scoring='neg_mean_squared_error', cv=kf)
lr_scores_mae = -cross_val_score(lr_model, X, y, scoring='neg_mean_absolute_error', cv=kf)

# Metrics
lr_mse = lr_scores_mse.mean()
lr_rmse = lr_mse ** 0.5
lr_mae = lr_scores_mae.mean()
lr_std = lr_scores_mse.std()
print(f"Linear Regression: MSE={lr_mse:.4f}, MAE={lr_mae:.4f}, RMSE={lr_rmse:.4f}, Std Dev={lr_std:.4f}")


Linear Regression: MSE=0.0000, MAE=0.0000, RMSE=0.0000, Std Dev=0.0000


# 2. ARIMA

In [10]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Fit ARIMA model
arima_model = ARIMA(y, order=(5, 1, 0))
arima_fitted = arima_model.fit()

# Predictions
y_pred_arima = arima_fitted.predict(start=1, end=len(y))
arima_mse = mean_squared_error(y[1:], y_pred_arima[1:])
arima_mae = mean_absolute_error(y[1:], y_pred_arima[1:])
arima_rmse = arima_mse ** 0.5
arima_std = np.std(y_pred_arima - y[1:])

print(f"ARIMA: MSE={arima_mse:.4f}, MAE={arima_mae:.4f}, RMSE={arima_rmse:.4f}, Std Dev={arima_std:.4f}")


ARIMA: MSE=0.2057, MAE=0.1885, RMSE=0.4535, Std Dev=4.4443


# 3. Support Vector Machines (SVM)

In [11]:
from sklearn.svm import SVR

# Initialize SVM model
svm_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# Perform 5-fold cross-validation
svm_scores_mse = -cross_val_score(svm_model, X, y, scoring='neg_mean_squared_error', cv=kf)
svm_scores_mae = -cross_val_score(svm_model, X, y, scoring='neg_mean_absolute_error', cv=kf)

# Metrics
svm_mse = svm_scores_mse.mean()
svm_rmse = svm_mse ** 0.5
svm_mae = svm_scores_mae.mean()
svm_std = svm_scores_mse.std()
print(f"SVM: MSE={svm_mse:.4f}, MAE={svm_mae:.4f}, RMSE={svm_rmse:.4f}, Std Dev={svm_std:.4f}")


SVM: MSE=509.0369, MAE=6.0592, RMSE=22.5618, Std Dev=120.5321


# 4. Random Forest

In [12]:
from sklearn.ensemble import RandomForestRegressor

# Initialize Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform 5-fold cross-validation
rf_scores_mse = -cross_val_score(rf_model, X, y, scoring='neg_mean_squared_error', cv=kf)
rf_scores_mae = -cross_val_score(rf_model, X, y, scoring='neg_mean_absolute_error', cv=kf)

# Metrics
rf_mse = rf_scores_mse.mean()
rf_rmse = rf_mse ** 0.5
rf_mae = rf_scores_mae.mean()
rf_std = rf_scores_mse.std()
print(f"Random Forest: MSE={rf_mse:.4f}, MAE={rf_mae:.4f}, RMSE={rf_rmse:.4f}, Std Dev={rf_std:.4f}")


Random Forest: MSE=0.3820, MAE=0.1696, RMSE=0.6181, Std Dev=0.1145


# 5. Long Short-Term Memory (LSTM)


In [13]:
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

lstm_mse_scores = []
lstm_mae_scores = []

# Perform cross-validation
for train_index, test_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y_scaled[train_index], y_scaled[test_index]

    # Reshape for LSTM
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

    # Build LSTM model
    lstm_model = Sequential([
        LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])
    lstm_model.compile(optimizer='adam', loss='mse')

    # Train the model
    lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    y_pred_lstm = lstm_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred_lstm)
    mae = mean_absolute_error(y_test, y_pred_lstm)
    lstm_mse_scores.append(mse)
    lstm_mae_scores.append(mae)

# Metrics
lstm_mse = np.mean(lstm_mse_scores)
lstm_mae = np.mean(lstm_mae_scores)
lstm_std = np.std(lstm_mse_scores)
lstm_rmse = np.sqrt(lstm_mse)

print(f"LSTM: MSE={lstm_mse:.4f}, MAE={lstm_mae:.4f}, RMSE={lstm_rmse:.4f}, Std Dev={lstm_std:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
LSTM: MSE=0.0000, MAE=0.0017, RMSE=0.0037, Std Dev=0.0000


# 6. Artificial Neural Networks (ANN)

In [14]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Check and print the shape of the data
print(f"X_train shape before reshape: {X_train.shape}")
print(f"X_test shape before reshape: {X_test.shape}")

# Reshape only if needed
if len(X_train.shape) == 3:
    X_train = tf.reshape(X_train, (X_train.shape[0], X_train.shape[-1]))
    X_test = tf.reshape(X_test, (X_test.shape[0], X_test.shape[-1]))

# Verify shapes after reshaping
print(f"X_train shape after reshape: {X_train.shape}")
print(f"X_test shape after reshape: {X_test.shape}")

# Build ANN model
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    Dense(32, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
ann_model.compile(optimizer='adam', loss='mse')

# Train the model
ann_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predictions
y_pred_ann = ann_model.predict(X_test)

# Metrics
ann_mse = mean_squared_error(y_test, y_pred_ann)
ann_mae = mean_absolute_error(y_test, y_pred_ann)
ann_rmse = ann_mse ** 0.5
ann_std = np.std(y_pred_ann)  # Calculate standard deviation of predictions

# Print metrics
print(f"ANN: MSE={ann_mse:.4f}, RMSE={ann_rmse:.4f}, MAE={ann_mae:.4f}, Std Dev={ann_std:.4f}")


X_train shape before reshape: (3668, 1, 10)
X_test shape before reshape: (916, 1, 10)
X_train shape after reshape: (3668, 10)
X_test shape after reshape: (916, 10)
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0491
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4192e-05
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1154e-05
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1172e-05
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.2007e-06
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.5690e-06
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.7746e-06
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1968e-06
Epoch 9/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3120e-06
Epoch 10/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 7.2473e-06
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
ANN: MSE=0.0000, RMSE=0.0023, MAE=0.0013, Std Dev=0.2568


# Hyperparameter Tuning

**LSTM Hyperparameter Tuning**

In [27]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Split and scale data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)  # Scale features
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))  # Scale target

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape data for LSTM (adding a third dimension)
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define hyperparameter combinations
hyperparams = [
    {"units": 50, "learning_rate": 0.001, "batch_size": 16, "epochs": 20},
    {"units": 100, "learning_rate": 0.001, "batch_size": 32, "epochs": 50},
    {"units": 100, "learning_rate": 0.01, "batch_size": 16, "epochs": 20},
]

best_mse = float("inf")
best_config = {}

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Iteratively evaluate hyperparameters
for params in hyperparams:
    mse_scores = []
    mae_scores = []

    for train_idx, val_idx in kf.split(X_train_lstm):
        X_train_fold, X_val_fold = X_train_lstm[train_idx], X_train_lstm[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # Build LSTM model
        model = Sequential([
            LSTM(params["units"], activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
            Dense(1)
        ])
        optimizer = Adam(learning_rate=params["learning_rate"])
        model.compile(optimizer=optimizer, loss='mse')

        # Train the model
        model.fit(X_train_fold, y_train_fold, epochs=params["epochs"], batch_size=params["batch_size"], verbose=0)

        # Predict and evaluate
        predictions = model.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, predictions)
        mae = mean_absolute_error(y_val_fold, predictions)

        mse_scores.append(mse)
        mae_scores.append(mae)

    # Calculate metrics
    mean_mse = np.mean(mse_scores)
    std_mse = np.std(mse_scores)
    mean_rmse = np.sqrt(mean_mse)
    mean_mae = np.mean(mae_scores)

    # Round metrics to 5 decimal places
    mean_mse = round(mean_mse, 5)
    std_mse = round(std_mse, 5)
    mean_rmse = round(mean_rmse, 5)
    mean_mae = round(mean_mae, 5)

    # Update best configuration
    if mean_mse < best_mse:
        best_mse = mean_mse
        best_config = {
            **params,
            "Mean MSE": mean_mse,
            "Std Dev (MSE)": std_mse,
            "Mean RMSE": mean_rmse,
            "Mean MAE": mean_mae,
        }

# Output the best configuration
print("Best LSTM Configuration:", best_config)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Best LSTM Configuration: {'units': 100, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 50, 'Mean MSE': 0.0, 'Std Dev (MSE)': 0.0, 'Mean RMSE': 0.00211, 'Mean MAE': 0.00135}


**ANN Hyperparameter Tuning**

In [26]:
# Ensure X_train and y_train are NumPy arrays
X_train_np = X_train.numpy() if hasattr(X_train, 'numpy') else X_train
y_train_np = y_train.numpy() if hasattr(y_train, 'numpy') else y_train

# Predefined hyperparameter combinations
ann_hyperparams = [
    {"hidden_layers": 1, "neurons": 64, "activation": 'relu', "learning_rate": 0.001, "batch_size": 16, "epochs": 20},
    {"hidden_layers": 2, "neurons": 128, "activation": 'relu', "learning_rate": 0.001, "batch_size": 32, "epochs": 50},
]

best_ann_mse = float("inf")
best_ann_config = {}

for params in ann_hyperparams:
    mse_scores = []
    mae_scores = []

    for train_idx, val_idx in kf.split(X_train_np, y_train_np):
        X_train_fold, X_val_fold = X_train_np[train_idx], X_train_np[val_idx]
        y_train_fold, y_val_fold = y_train_np[train_idx], y_train_np[val_idx]

        # Build ANN model
        model = Sequential()
        for _ in range(params["hidden_layers"]):
            model.add(Dense(params["neurons"], activation=params["activation"], input_dim=X_train.shape[1]))
        model.add(Dense(1))
        optimizer = Adam(learning_rate=params["learning_rate"])
        model.compile(optimizer=optimizer, loss='mse')

        # Train the model
        model.fit(X_train_fold, y_train_fold, epochs=params["epochs"], batch_size=params["batch_size"], verbose=0)

        # Predict and evaluate
        predictions = model.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, predictions)
        mae = mean_absolute_error(y_val_fold, predictions)

        mse_scores.append(mse)
        mae_scores.append(mae)

    # Calculate metrics
    mean_mse = np.mean(mse_scores)
    std_mse = np.std(mse_scores)
    mean_rmse = np.sqrt(mean_mse)
    mean_mae = np.mean(mae_scores)

    # Round metrics to 5 decimal places
    mean_mse = round(mean_mse, 5)
    std_mse = round(std_mse, 5)
    mean_rmse = round(mean_rmse, 5)
    mean_mae = round(mean_mae, 5)

    # Update best configuration
    if mean_mse < best_ann_mse:
        best_ann_mse = mean_mse
        best_ann_config = {
            **params,
            "Mean MSE": mean_mse,
            "Std Dev (MSE)": std_mse,
            "Mean RMSE": mean_rmse,
            "Mean MAE": mean_mae,
        }

print("Best ANN Configuration:", best_ann_config)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Best ANN Configuration: {'hidden_layers': 2, 'neurons': 128, 'activation': 'relu', 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 50, 'Mean MSE': 0.0, 'Std Dev (MSE)': 0.0, 'Mean RMSE': 0.00157, 'Mean MAE': 0.00099}


**Random Forest Hyperparameter Tuning**

In [28]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Ensure X_train and y_train are NumPy arrays
X_train_np = X_train.numpy() if hasattr(X_train, 'numpy') else X_train
y_train_np = y_train.numpy() if hasattr(y_train, 'numpy') else y_train

# Predefined hyperparameter grid
rf_hyperparams = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_leaf': [2]
}

# Grid search with 5-fold cross-validation
rf_model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=rf_hyperparams,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=0
)
grid_search.fit(X_train_np, y_train_np)

# Evaluate the best model
rf_best_model = grid_search.best_estimator_
rf_predictions = rf_best_model.predict(X_test)

# Metrics
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_std = np.std(y_test - rf_predictions)  # Standard deviation of residuals

# Round metrics to 5 decimal places
rf_mse = round(rf_mse, 5)
rf_rmse = round(rf_rmse, 5)
rf_mae = round(rf_mae, 5)
rf_std = round(rf_std, 5)

# Best hyperparameters and metrics
rf_best_config = {
    "Best Parameters": grid_search.best_params_,
    "Mean MSE": rf_mse,
    "Mean RMSE": rf_rmse,
    "Mean MAE": rf_mae,
    "Std Dev": rf_std
}
print("Best Random Forest Configuration:", rf_best_config)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

Best Random Forest Configuration: {'Best Parameters': {'max_depth': 20, 'min_samples_leaf': 2, 'n_estimators': 200}, 'Mean MSE': 0.0, 'Mean RMSE': 0.0009, 'Mean MAE': 0.00024, 'Std Dev': 0.37813}
